In [ ]:
import numpy as np
import cv2
import os
import math


In [ ]:
def preprocess_image(image_name):
    
    image = cv2.imread(image_name, cv2.IMREAD_GRAYSCALE)  # Read the image as grayscale

    histogram, _ = np.histogram(image, bins=255, range=[0, 255])
    
    pixels = 0
    mid_index = 0
    for i in histogram:
        pixels += i
        if pixels >= 204800:
            break
        mid_index += 1
            
        
    peak_index = -1
    peak_level_number = -1
    for i in range(0, mid_index):
        if histogram[i] > peak_level_number:
            peak_index = i
            peak_level_number = histogram[i]


    max_point = (peak_index, peak_level_number)
    min_point_left = (0, histogram[0])
    min_point_right = (mid_index, histogram[mid_index])

    # Calculate the slope of a line
    slope = (max_point[1] - min_point_left[1]) / (max_point[0] - min_point_left[0] + 0.000001)
    max_deviation = -1
    threshold = 0
    # Calculate the deviation of each gray level on the histogram and find the threshold corresponding to the maximum deviation
    for i in range(0, peak_index):
        deviation = slope * i - histogram[i]
        if deviation > max_deviation:
            max_deviation = deviation
            threshold = i
    slope = (min_point_right[1] - max_point[1]) / (max_point[0] - min_point_right[0] + 0.000001)

    # Calculate the deviation of each gray level on the histogram and find the threshold corresponding to the maximum deviation
    for i in range(peak_index, mid_index):
        deviation = histogram[peak_index] +  slope * (i-peak_index) - histogram[i]
        if deviation > max_deviation:
            max_deviation = deviation
            threshold = i
    
    peak_index = -1
    peak_level_number = -1
    for i in range(mid_index, 254):
        if histogram[i] > peak_level_number:
            peak_index = i
            peak_level_number = histogram[i]


    max_point = (peak_index, peak_level_number)
    min_point_left = (mid_index, histogram[mid_index])
    min_point_right = (254, histogram[254])

    # Calculate the slope of a line
    slope = (max_point[1] - min_point_left[1]) / (max_point[0] - min_point_left[0] + 0.000001)
    max_deviation = -1
    threshold2 = 0

    # Calculate the deviation of each gray level on the histogram and find the threshold corresponding to the maximum deviation

    for i in range(mid_index, peak_index):
        deviation = slope * i - histogram[i]
        if deviation > max_deviation:
            max_deviation = deviation
            threshold2 = i
            
    slope = (min_point_right[1] - max_point[1]) / (max_point[0] - min_point_right[0] + 0.000001)
    # Calculate the deviation of each gray level on the histogram and find the threshold corresponding to the maximum deviation
    for i in range(peak_index, 254):
        deviation = histogram[peak_index] +  slope * (i-peak_index) - histogram[i]
        if deviation > max_deviation:
            max_deviation = deviation
            threshold2 = i
    
    if mid_index < 50 or mid_index > 200:
        threshold = 85
        threshold2 = 200
    
    if histogram[peak_index] < 7000:
        threshold = 85
        threshold2 = 200

    image[image < threshold] = 0
    image[(image >= threshold) & (image <= threshold2)] = 127
    image[image > threshold2] = 255

    
    return image


In [ ]:
folder_path = "../train/preprocess"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")

folder_path = "../valid/preprocess"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")

In [ ]:
folder_name = "../train/preprocess"
filenames = os.listdir("../train/train/")
filenames.sort()

number = 0
for filename in filenames:
    
    image = preprocess_image(f"../train/train/{filename}")
    output_path = f"{folder_name}/{filename}.jpg"
    cv2.imwrite(output_path, image)
    print(f"image {number}.jpg is already process")
    number += 1


In [ ]:
folder_name = "../valid/preprocess"
filenames = os.listdir("../valid/valid/")

number = 0
for filename in filenames:
    
    image = preprocess_image(f"../valid/valid/{filename}")
    output_path = f"{folder_name}/{filename}.jpg"
    cv2.imwrite(output_path, image)
    print(f"image {number}.jpg is already process")
    number += 1
